<a href="https://colab.research.google.com/github/saiparameshwarrao/NLP/blob/main/Assignment-8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Example text data (you can replace this with any larger corpus) text = """ Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her. [CO5]
#### (i) Build the Transformer Model on above dataset

#### (ii) Train the model using 20, 60, 70 epochs

 ####  (iii) After training, use the model to generate new text by feeding it an initial seed text

####  (iv) Experimenting and Improving the Model by large dataset and hyper tune parameter.

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, MultiHeadAttention, LayerNormalization, Dense, Dropout
from tensorflow.keras import Sequential, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

vocab_size = 1000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts([text])

sequences = tokenizer.texts_to_sequences([text])[0]

input_sequences = []
for i in range(1, len(sequences)):
    input_sequences.append(sequences[:i+1])

max_sequence_len = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

X, y = input_sequences[:,:-1], input_sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

class TransformerModel(Model):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_dim):
        super(TransformerModel, self).__init__()
        self.embedding = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization()
        self.layernorm2 = LayerNormalization()
        self.dropout1 = Dropout(0.1)
        self.dropout2 = Dropout(0.1)
        self.final_layer = Dense(vocab_size, activation="softmax")

    def call(self, inputs):
        x = self.embedding(inputs)
        attn_output = self.attention(x, x)
        x = self.layernorm1(x + self.dropout1(attn_output))
        ffn_output = self.dense_proj(x)
        x = self.layernorm2(x + self.dropout2(ffn_output))

        return self.final_layer(x[:, -1])

embed_dim = 128
num_heads = 4
ff_dim = 512

transformer_model = TransformerModel(vocab_size, embed_dim, num_heads, ff_dim)
transformer_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

epochs_list = [20,60,70]
for epochs in epochs_list:
    print(f"\nTraining model with {epochs} epochs:")
    transformer_model.fit(X, y, epochs=epochs, verbose=1)

def generate_text(model, tokenizer, seed_text, max_length=50):
    for _ in range(max_length):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1).item()
        output_word = tokenizer.index_word.get(predicted_word_index, "")
        seed_text += " " + output_word
    return seed_text

seed_text = "Once upon a time"
print("\nGenerated text:", generate_text(transformer_model, tokenizer, seed_text))


Training model with 20 epochs:
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 140ms/step - accuracy: 0.0000e+00 - loss: 6.9420
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - accuracy: 0.2496 - loss: 5.8386
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.3778 - loss: 5.2432
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.4246 - loss: 4.7474
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.4956 - loss: 4.2064
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.6898 - loss: 3.6831
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.7591 - loss: 3.2224
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.7937 - loss: 2.8000
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.7955 - loss: 2.4268
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.7504 - loss: 2.1023
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.7729 - loss: 1.8192
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0